# Model WO factor

Att göra 

Slå ut PiT WO för att kunna tracka den

RemainingCapital , om detta join finns också så modi ,  för de andra säg global 


1. modulera båda? måste jag göra i datat oavsett
2. varför aviker forecast från actual när det är på månader som finns? 

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier , GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold , cross_validate, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import accuracy_score ,mean_squared_error
from sklearn.preprocessing import LabelEncoder
import os

import pandas as pd
import numpy as np


I need python version 3.9.17 in order to run 
import pymssql

In [2]:
# from Preprocessing_Application import DataPreprocessor

# server = 'reporting-db.nystartfinans.net'
# database = 'reporting-db'
# username = 'Andreas'
# password = 'nCq8Sg@1lYnd(E'

# # /Users/andreasnilsson/Library/CloudStorage/OneDrive-Nstart/Skrivbordet/Repository Homes/GWO Portfolio Modeling/SQL


# path = "/Users/andreasnilsson/Desktop/Repository Homes/GWO Portfolio Modeling/SQL/Gross WO & Ever90 & RunOff Monthly.sql"

# preprocessor = DataPreprocessor(server, database, username, password)

# df = preprocessor.fetch_data_from_sql(path)



In [4]:

# Save the DataFrame to a CSV file
csv_file = 'HistoricalSnapshot.csv'
#df.to_csv(csv_file, index=False)  # Set index=False to exclude the DataFrame index


# Load the CSV file back into a DataFrame
df = pd.read_csv(csv_file)





In [5]:
df = df[(df.DisbursedDate > '2022-01-01') & (df.DisbursedDate < '2023-01-01')]

In [6]:

df['DisbursedDate'] = pd.to_datetime(df['DisbursedDate'], errors='coerce')


## Import UCblanco data

In [7]:
bla = pd.read_excel('UCBLANCO SCORECARD 19F.xlsx')

print(len(bla))

bla['Key'] = bla.apply(lambda row:  str(row.Amount)  + str(row.Inquiries12M) +str(row.Applicationtype) +str(row.ReceivedDate), axis=1)  #  


blanco = bla[['Key','Probability_model','Probability_Bins','Applicationtype']]
blanco

9129


,Key,Probability_model,Probability_Bins,Applicationtype
0,90000902019-01-02 00:00:00,0.027553,0,0
1,3100003722019-01-02 00:00:00,0.075324,3,2
2,1200001102019-01-02 00:00:00,0.077158,3,0
3,700001202019-01-02 00:00:00,0.134075,4,0
4,80000702019-01-03 00:00:00,0.026490,0,0
...,...,...,...,...
9124,3000005002022-07-19 00:00:00,0.129115,4,0
9125,1100001502022-07-19 00:00:00,0.051345,3,0
9126,2000002802022-07-19 00:00:00,0.209211,5,0
9127,3000002102022-07-19 00:00:00,0.125784,4,0


## import previous data to join

In [8]:
frame = pd.read_excel('PDTo23-12.xlsx')

print(len(frame))

frame['Key'] = frame.apply(lambda row: str(row.Amount)  + str(row.Inquiries12M)  +str(row.Applicationtype)+str(row.ReceivedDate) ,axis=1)  #  +  +str(row.Applicationtype) 

frame

12697


,SSN,UCScore,age,Inquiries12M,UtilizationRatio,Amount,MaritalStatus,ReceivedDate,Applicationtype,Ever90,...,PaymentRemarksNo,score,old_model,Z,P,Z_calibrated,P_calibrated,Probability_Bins,YearMonth,Key
0,Unknown,0.016,46,6,9.35,155000,PARTNER,2018-01-02,2,0,...,0,-5.0003,0.669086,-0.900782,0.288890,-3.243715,0.037553,0,2018-01,155000622018-01-02 00:00:00
1,Unknown,0.412,32,8,-100.00,175000,PARTNER,2018-01-02,1,0,...,3,-4.4532,1.150730,0.279015,0.569305,-2.183408,0.101250,4,2018-01,175000812018-01-02 00:00:00
2,230AEAF422AA2FA458E7C6F4A7049238392971563D27BF...,0.423,54,10,-100.00,180000,MARRIED,2018-01-02,0,0,...,4,-5.0003,0.669086,-0.110394,0.472430,-2.549713,0.072446,3,2018-01,1800001002018-01-02 00:00:00
3,3FA4E5ECA51A8357E398F8C1213561EED2057484087ED5...,0.131,43,5,-100.00,50000,MARRIED,2018-01-02,0,0,...,11,-5.0003,0.669086,-0.635105,0.346354,-3.026432,0.046246,1,2018-01,50000502018-01-02 00:00:00
4,BFFB31FD45F729F65A29375283FBB2302F396DEDA0FF8C...,0.449,56,11,62.60,85000,SINGLE,2018-01-03,0,0,...,0,-4.6521,0.945136,-0.067081,0.483236,-2.508852,0.075240,3,2018-01,850001102018-01-03 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12692,Unknown,0.010,63,3,31.82,55000,PARTNER,2023-12-20,1,0,...,0,-5.0003,0.669086,-1.385638,0.200105,-3.579429,0.027135,0,2023-12,55000312023-12-20 00:00:00
12693,Unknown,0.009,45,3,-100.00,135000,SINGLE,2023-12-21,2,0,...,0,-4.6521,0.945136,-1.289922,0.215866,-3.519833,0.028753,0,2023-12,135000322023-12-21 00:00:00
12694,826DBC4BA358FA7B7C65906E5AF3DEEF98D9072CB04381...,0.091,39,8,94.93,190000,PARTNER,2023-12-26,0,0,...,6,-5.0003,0.669086,-0.527432,0.371116,-2.932801,0.050556,1,2023-12,190000802023-12-26 00:00:00
12695,42D1516147D5403980E8EF457B9D03B3F191204C05D653...,0.160,36,11,97.57,85000,SINGLE,2023-12-28,0,0,...,0,-4.1050,1.622251,-0.219560,0.445329,-2.652184,0.065854,2,2023-12,850001102023-12-28 00:00:00


In [9]:
frame[frame.AccountNumber == 7781222]

,SSN,UCScore,age,Inquiries12M,UtilizationRatio,Amount,MaritalStatus,ReceivedDate,Applicationtype,Ever90,...,PaymentRemarksNo,score,old_model,Z,P,Z_calibrated,P_calibrated,Probability_Bins,YearMonth,Key
9349,DD46CBDF34DC432EB3CA22C1C96C4CEBF7FB4212A599D2...,0.026,22,5,-100.0,85000,PARTNER,2022-07-18,0,0,...,0,-4.2059,1.46884,-0.391015,0.403473,-2.810452,0.056762,2,2022-07,85000502022-07-18 00:00:00


In [10]:
f18 = frame[frame.ReceivedDate == '2022-07-18']
b18 = bla[bla.ReceivedDate == '2022-07-18']

hit = pd.merge(f18[['Key','AccountNumber']],b18, on='Key', how='outer')

hit[hit.Inquiries12M.isna()  ]

,Key,AccountNumber,Unnamed: 0,ucrbv2,age,Inquiries12M,Applicationtype,Amount,ReceivedDate,Ever90,YearMonth,Probability_model_n,Probability_model,Probability_Bins,Year,Month


In [11]:
print(len(f18))
f18

10


,SSN,UCScore,age,Inquiries12M,UtilizationRatio,Amount,MaritalStatus,ReceivedDate,Applicationtype,Ever90,...,PaymentRemarksNo,score,old_model,Z,P,Z_calibrated,P_calibrated,Probability_Bins,YearMonth,Key
9340,6A740D5A58370273657D45CEA27E536F7E0A4077A20978...,0.072,26,8,-100.00,135000,SINGLE,2022-07-18,0,1,...,0,-4.1050,1.622251,-0.282638,0.429807,-2.710877,0.062335,2,2022-07,135000802022-07-18 00:00:00
9341,F0FE4F2A5066F30890A938B43FA2A0705DED6B2191354A...,0.083,36,25,-100.00,158000,SINGLE,2022-07-18,0,1,...,0,-4.1050,1.622251,0.099212,0.524783,-2.351755,0.086926,3,2022-07,1580002502022-07-18 00:00:00
9342,FABDDB92FE173CEB1DFFDE1C6022205027173A7A8F5FD2...,0.183,23,19,-100.00,185000,PARTNER,2022-07-18,0,0,...,1,-4.2059,1.468840,0.382551,0.594488,-2.088185,0.110251,4,2022-07,1850001902022-07-18 00:00:00
9343,16BA5893DABFC99DC67F5C603A50C7469A0AD5EC02F46B...,0.055,24,5,99.02,125000,PARTNER,2022-07-18,0,0,...,0,-4.2059,1.468840,-0.375563,0.407197,-2.796369,0.057521,2,2022-07,125000502022-07-18 00:00:00
9344,EC7F7E03EC76ED87F25DFCA9CA4ACCB04EEB71855A58D7...,0.054,51,2,0.00,300000,SINGLE,2022-07-18,0,0,...,6,-4.6521,0.945136,-1.068307,0.255725,-3.369117,0.033275,0,2022-07,300000202022-07-18 00:00:00
9345,BE92876EB34442E0D88AA8C494791BBA9303541174ABD0...,0.285,33,18,81.58,300000,SINGLE,2022-07-18,0,0,...,0,-4.1050,1.622251,0.338256,0.583767,-2.128724,0.106336,4,2022-07,3000001802022-07-18 00:00:00
9346,E76766CD2050D30D4937C7C748DDB280FA7813FDB18657...,0.049,38,3,98.66,300000,PARTNER,2022-07-18,0,0,...,0,-4.4532,1.150730,-0.761026,0.318423,-3.132042,0.041805,1,2022-07,300000302022-07-18 00:00:00
9347,D4AB264C12B7C8F2DD91464D69FBA34130C20B719D03BB...,0.070,21,8,-100.00,65000,SINGLE,2022-07-18,0,0,...,0,-3.8577,2.067983,-0.177676,0.455698,-2.612980,0.068308,2,2022-07,65000802022-07-18 00:00:00
9348,D11909D321191321CA77300A738C376695DE5E767D8D53...,0.148,27,9,96.41,255000,SINGLE,2022-07-18,0,0,...,0,-4.1050,1.622251,-0.115680,0.471112,-2.554695,0.072112,3,2022-07,255000902022-07-18 00:00:00
9349,DD46CBDF34DC432EB3CA22C1C96C4CEBF7FB4212A599D2...,0.026,22,5,-100.00,85000,PARTNER,2022-07-18,0,0,...,0,-4.2059,1.468840,-0.391015,0.403473,-2.810452,0.056762,2,2022-07,85000502022-07-18 00:00:00


In [12]:
print(len(b18))
b18

10


,Unnamed: 0,ucrbv2,age,Inquiries12M,Applicationtype,Amount,ReceivedDate,Ever90,YearMonth,Probability_model_n,Probability_model,Probability_Bins,Year,Month,Key
9108,5366,0.317046,25,8,0,135000,2022-07-18,1,2022-07,0.575538,0.098427,4,2022,7,135000802022-07-18 00:00:00
9109,3196,0.202743,36,25,0,158000,2022-07-18,1,2022-07,0.498276,0.071924,3,2022,7,1580002502022-07-18 00:00:00
9110,11030,0.294402,23,19,0,185000,2022-07-18,0,2022-07,0.611155,0.113361,4,2022,7,1850001902022-07-18 00:00:00
9111,3327,0.170244,21,8,0,65000,2022-07-18,0,2022-07,0.508229,0.074927,3,2022,7,65000802022-07-18 00:00:00
9112,9155,0.121917,51,2,0,300000,2022-07-18,0,2022-07,0.289301,0.029761,1,2022,7,300000202022-07-18 00:00:00
9113,1698,0.135843,23,5,0,125000,2022-07-18,0,2022-07,0.463984,0.062410,3,2022,7,125000502022-07-18 00:00:00
9114,9854,0.279656,32,18,0,300000,2022-07-18,0,2022-07,0.545720,0.087299,4,2022,7,3000001802022-07-18 00:00:00
9115,6170,0.159061,27,9,0,255000,2022-07-18,0,2022-07,0.468542,0.063603,3,2022,7,255000902022-07-18 00:00:00
9116,624,0.152615,21,5,0,85000,2022-07-18,0,2022-07,0.486531,0.068524,3,2022,7,85000502022-07-18 00:00:00
9117,4765,0.136817,38,3,0,300000,2022-07-18,0,2022-07,0.369838,0.042001,2,2022,7,300000302022-07-18 00:00:00


In [13]:
hit[hit.Inquiries12M> -1  ]

,Key,AccountNumber,Unnamed: 0,ucrbv2,age,Inquiries12M,Applicationtype,Amount,ReceivedDate,Ever90,YearMonth,Probability_model_n,Probability_model,Probability_Bins,Year,Month
0,135000802022-07-18 00:00:00,7780950,5366,0.317046,25,8,0,135000,2022-07-18,1,2022-07,0.575538,0.098427,4,2022,7
1,1580002502022-07-18 00:00:00,7786908,3196,0.202743,36,25,0,158000,2022-07-18,1,2022-07,0.498276,0.071924,3,2022,7
2,1850001902022-07-18 00:00:00,7780778,11030,0.294402,23,19,0,185000,2022-07-18,0,2022-07,0.611155,0.113361,4,2022,7
3,125000502022-07-18 00:00:00,7780794,1698,0.135843,23,5,0,125000,2022-07-18,0,2022-07,0.463984,0.062410,3,2022,7
4,300000202022-07-18 00:00:00,7780877,9155,0.121917,51,2,0,300000,2022-07-18,0,2022-07,0.289301,0.029761,1,2022,7
5,3000001802022-07-18 00:00:00,7782857,9854,0.279656,32,18,0,300000,2022-07-18,0,2022-07,0.545720,0.087299,4,2022,7
6,300000302022-07-18 00:00:00,7782477,4765,0.136817,38,3,0,300000,2022-07-18,0,2022-07,0.369838,0.042001,2,2022,7
7,65000802022-07-18 00:00:00,7781230,3327,0.170244,21,8,0,65000,2022-07-18,0,2022-07,0.508229,0.074927,3,2022,7
8,255000902022-07-18 00:00:00,7782238,6170,0.159061,27,9,0,255000,2022-07-18,0,2022-07,0.468542,0.063603,3,2022,7
9,85000502022-07-18 00:00:00,7781222,624,0.152615,21,5,0,85000,2022-07-18,0,2022-07,0.486531,0.068524,3,2022,7


In [14]:
pd.merge(f18[['Key','AccountNumber']],b18, on='Key', how='outer')

,Key,AccountNumber,Unnamed: 0,ucrbv2,age,Inquiries12M,Applicationtype,Amount,ReceivedDate,Ever90,YearMonth,Probability_model_n,Probability_model,Probability_Bins,Year,Month
0,135000802022-07-18 00:00:00,7780950,5366,0.317046,25,8,0,135000,2022-07-18,1,2022-07,0.575538,0.098427,4,2022,7
1,1580002502022-07-18 00:00:00,7786908,3196,0.202743,36,25,0,158000,2022-07-18,1,2022-07,0.498276,0.071924,3,2022,7
2,1850001902022-07-18 00:00:00,7780778,11030,0.294402,23,19,0,185000,2022-07-18,0,2022-07,0.611155,0.113361,4,2022,7
3,125000502022-07-18 00:00:00,7780794,1698,0.135843,23,5,0,125000,2022-07-18,0,2022-07,0.463984,0.062410,3,2022,7
4,300000202022-07-18 00:00:00,7780877,9155,0.121917,51,2,0,300000,2022-07-18,0,2022-07,0.289301,0.029761,1,2022,7
5,3000001802022-07-18 00:00:00,7782857,9854,0.279656,32,18,0,300000,2022-07-18,0,2022-07,0.545720,0.087299,4,2022,7
6,300000302022-07-18 00:00:00,7782477,4765,0.136817,38,3,0,300000,2022-07-18,0,2022-07,0.369838,0.042001,2,2022,7
7,65000802022-07-18 00:00:00,7781230,3327,0.170244,21,8,0,65000,2022-07-18,0,2022-07,0.508229,0.074927,3,2022,7
8,255000902022-07-18 00:00:00,7782238,6170,0.159061,27,9,0,255000,2022-07-18,0,2022-07,0.468542,0.063603,3,2022,7
9,85000502022-07-18 00:00:00,7781222,624,0.152615,21,5,0,85000,2022-07-18,0,2022-07,0.486531,0.068524,3,2022,7


In [15]:
reborn = pd.merge(frame[['Key','AccountNumber']],blanco, on='Key', how='inner')

# Import PD 

In [16]:
## Merge
df = pd.merge( reborn ,df, on = 'AccountNumber')
df

,Key,AccountNumber,Probability_model,Probability_Bins,Applicationtype,mob,DisbursedDate,SnapshotDate,Amount,Ever90,...,WO_flag,RemainingCapital,PurchasePrice,maxAmount,MaxAmountAdjusted,CurrentAmountAdjusted,WO_Amount,run_off,WO_Amount2,WO_Amount_PiT
0,500000-122021-10-28 00:00:00,7685589,0.022925,0,2,0,2022-01-25,2022-01-31,500000.00,0,...,0,NaN,NaN,500000.00,500000.00,500000.00,NaN,1.000000,0.0,0.0
1,500000-122021-10-28 00:00:00,7685589,0.022925,0,2,1,2022-01-25,2022-02-28,499170.58,0,...,0,NaN,NaN,500000.00,500000.00,499170.58,NaN,0.998341,0.0,0.0
2,500000-122021-10-28 00:00:00,7685589,0.022925,0,2,2,2022-01-25,2022-03-31,498331.52,0,...,0,NaN,NaN,500000.00,500000.00,498331.52,NaN,0.996663,0.0,0.0
3,500000-122021-10-28 00:00:00,7685589,0.022925,0,2,3,2022-01-25,2022-04-30,497482.70,0,...,0,NaN,NaN,500000.00,500000.00,497482.70,NaN,0.994965,0.0,0.0
4,500000-122021-10-28 00:00:00,7685589,0.022925,0,2,4,2022-01-25,2022-05-31,496624.02,0,...,0,NaN,NaN,500000.00,500000.00,496624.02,NaN,0.993248,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52597,1950001302022-10-31 00:00:00,7829872,0.047206,2,0,9,2022-11-03,2023-08-31,187774.11,0,...,0,NaN,NaN,194360.22,194360.22,187774.11,NaN,0.966114,0.0,0.0
52598,1950001302022-10-31 00:00:00,7829872,0.047206,2,0,10,2022-11-03,2023-09-30,187065.28,0,...,0,NaN,NaN,194360.22,194360.22,187065.28,NaN,0.962467,0.0,0.0
52599,1950001302022-10-31 00:00:00,7829872,0.047206,2,0,11,2022-11-03,2023-10-31,186349.83,0,...,0,NaN,NaN,194360.22,194360.22,186349.83,NaN,0.958786,0.0,0.0
52600,1950001302022-10-31 00:00:00,7829872,0.047206,2,0,12,2022-11-03,2023-11-30,185627.71,0,...,0,NaN,NaN,194360.22,194360.22,185627.71,NaN,0.955070,0.0,0.0


In [17]:
max(df.DisbursedDate)

Timestamp('2022-12-19 00:00:00')

In [18]:
df[df.WO_Amount2 > 0].tail(30)

,Key,AccountNumber,Probability_model,Probability_Bins,Applicationtype,mob,DisbursedDate,SnapshotDate,Amount,Ever90,...,WO_flag,RemainingCapital,PurchasePrice,maxAmount,MaxAmountAdjusted,CurrentAmountAdjusted,WO_Amount,run_off,WO_Amount2,WO_Amount_PiT
50575,1450001302022-10-17 00:00:00,7823909,0.135769,4,0,10,2022-10-17,2023-08-31,0.00,1,...,0,124510.03,82165.62,125360.00,0.0,0.0,0.00,0.0,124510.03,0.00
50576,1450001302022-10-17 00:00:00,7823909,0.135769,4,0,11,2022-10-17,2023-09-30,0.00,1,...,0,124510.03,82165.62,125360.00,0.0,0.0,0.00,0.0,124510.03,0.00
50577,1450001302022-10-17 00:00:00,7823909,0.135769,4,0,12,2022-10-17,2023-10-31,0.00,1,...,0,124510.03,82165.62,125360.00,0.0,0.0,0.00,0.0,124510.03,0.00
50578,1450001302022-10-17 00:00:00,7823909,0.135769,4,0,13,2022-10-17,2023-11-30,0.00,1,...,0,124510.03,82165.62,125360.00,0.0,0.0,0.00,0.0,124510.03,0.00
50579,1450001302022-10-17 00:00:00,7823909,0.135769,4,0,14,2022-10-17,2023-12-31,0.00,1,...,0,124510.03,82165.62,125360.00,0.0,0.0,0.00,0.0,124510.03,0.00
51094,16000013302022-10-21 00:00:00,7825920,0.250949,5,0,12,2022-10-24,2023-10-31,0.00,1,...,1,159017.25,103618.99,160000.00,0.0,0.0,0.00,0.0,159017.25,159017.25
51095,16000013302022-10-21 00:00:00,7825920,0.250949,5,0,13,2022-10-24,2023-11-30,0.00,1,...,0,159017.25,103618.99,160000.00,0.0,0.0,0.00,0.0,159017.25,0.00
51096,16000013302022-10-21 00:00:00,7825920,0.250949,5,0,14,2022-10-24,2023-12-31,0.00,1,...,0,159017.25,103618.99,160000.00,0.0,0.0,0.00,0.0,159017.25,0.00
51147,300000402022-10-21 00:00:00,7826670,0.052865,3,0,8,2022-10-26,2023-06-30,0.00,1,...,1,295630.15,170904.03,299155.82,0.0,0.0,0.00,0.0,295630.15,295630.15
51148,300000402022-10-21 00:00:00,7826670,0.052865,3,0,9,2022-10-26,2023-07-31,0.00,1,...,0,295630.15,170904.03,299155.82,0.0,0.0,0.00,0.0,295630.15,0.00


In [19]:
df[df.AccountNumber == 7222664] # 5534920 7222664 7222003


df['CurrentAmountAdjusted'] = np.where(df.Ever90 == 0,df.CurrentAmount,0    )

In [20]:
df[df.AccountNumber == 7222003]

,Key,AccountNumber,Probability_model,Probability_Bins,Applicationtype,mob,DisbursedDate,SnapshotDate,Amount,Ever90,...,WO_flag,RemainingCapital,PurchasePrice,maxAmount,MaxAmountAdjusted,CurrentAmountAdjusted,WO_Amount,run_off,WO_Amount2,WO_Amount_PiT


In [21]:


#df['Row'] = 1 
df['DisbursedDate'] = pd.to_datetime(df['DisbursedDate'])


# Combine Year and Month into a 'YearMonth' column for grouping
df['YearMonth'] = df['DisbursedDate'].dt.to_period('M')

# Now group by 'mob', 'YearMonth' and then sum 'Row', 'CurrentAmount', 'Ever90', and 'WO_flag'
vintage_data = df.groupby(['Probability_Bins','Applicationtype','mob']).agg({ 'CurrentAmountAdjusted': 'sum' }).reset_index()

In [22]:
vintage_data

,Probability_Bins,Applicationtype,mob,CurrentAmountAdjusted
0,0,0,0,17004068.77
1,0,0,1,16197430.84
2,0,0,2,15764484.48
3,0,0,3,15479807.69
4,0,0,4,15074911.58
...,...,...,...,...
423,5,2,18,592921.50
424,5,2,19,601003.87
425,5,2,20,603449.26
426,5,2,21,616679.05


In [23]:
vintage_data.to_excel("Churn_Consolidate_RiskBins_Blanco_22_C.xlsx", index=False)

In [34]:
# vintage_data[vintage_data.Year == 2018].sort_values(by = ['Year','mob' ]).head(60)

In [9]:
# ## Controll Ever90 

# see2018_1_12 =  df[(df.Year == 2018) & ( df.Ever90 == 1  ) &  ( df.mob == 24  )  ]

# # print(len(see2018_1_12))

# see2018_1_136 = df[(df.Year == 2018) & ( df.Ever90 == 1  ) &  ( df.mob == 56  )  ]

# print(len(pd.merge(see2018_1_12, see2018_1_136, on='AccountNumber', how='inner')))

# ## Controll WO_flag 

# see2018_1_12 =  df[(df.Year == 2018) & ( df.WO_flag == 1  ) &  ( df.mob == 12  )  ]

# print(len(see2018_1_12))

# see2018_1_136 = df[(df.Year == 2018) & ( df.WO_flag == 1  ) &  ( df.mob == 46  )  ]

# intersection = pd.merge(see2018_1_12, see2018_1_136, on='AccountNumber', how='inner')

# print(len(intersection))

In [10]:
run_off = df.groupby(['WO_Sell_Split','mob', 'YearMonth']).agg({'run_off': 'mean'}).reset_index()
run_off



,WO_Sell_Split,mob,YearMonth,run_off
0,Global,0,2015-10,0.611111
1,Global,0,2015-11,0.750000
2,Global,0,2015-12,0.636364
3,Global,0,2016-01,0.876136
4,Global,0,2016-02,0.750000
...,...,...,...,...
7894,Modhi,80,2016-12,0.000000
7895,Modhi,80,2017-03,0.000000
7896,Modhi,81,2016-12,0.000000
7897,Modhi,82,2016-12,0.000000


In [11]:
all =  pd.merge(  vintage_data , run_off, on=['WO_Sell_Split','mob','YearMonth'], how='inner') 

## THIS BELLOW WILL NOT REFLECT THE TOTAL NOW SINCE I SPLIT ON MODI / GLOBAL

In [12]:
all['Ever90_%'] = all.Ever90/ all.Row 
all['WO_flag_%'] = all.WO_flag/ all.Row 

In [13]:
## to include columns to pivot in excel falls on the correct position/cells

syntetic = all[all.WO_Sell_Split ==  'Global'].copy()
syntetic['WO_Sell_Split'] = 'Modhi'
for i in syntetic.columns[3:] : syntetic[i] = 0

all = pd.concat((all, syntetic) ,axis=0) 

In [14]:
all.to_excel("WO_.xlsx", index=False)

In [15]:
all

,WO_Sell_Split,mob,YearMonth,Row,CurrentAmount,WO_Ever,Ever90,WO_flag,run_off,Ever90_%,WO_flag_%
0,Global,0,2015-10,18,4160000.00,0.0,0,0,0.611111,0.0,0.0
1,Global,0,2015-11,20,4758581.81,0.0,0,0,0.750000,0.0,0.0
2,Global,0,2015-12,11,2814717.00,0.0,0,0,0.636364,0.0,0.0
3,Global,0,2016-01,9,1553103.00,0.0,0,0,0.876136,0.0,0.0
4,Global,0,2016-02,24,5505101.00,0.0,0,0,0.750000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
5022,Modhi,102,2015-01,0,0.00,0.0,0,0,0.000000,0.0,0.0
5023,Modhi,103,2015-01,0,0.00,0.0,0,0,0.000000,0.0,0.0
5024,Modhi,104,2015-01,0,0.00,0.0,0,0,0.000000,0.0,0.0
5025,Modhi,105,2015-01,0,0.00,0.0,0,0,0.000000,0.0,0.0


,WO_Sell_Split,mob,YearMonth,Row,CurrentAmount,WO_Ever,Ever90,WO_flag,run_off,Ever90_%,WO_flag_%
0,Global,0,2015-10,18,4160000.00,0.0,0,0,0.611111,0.0,0.0
1,Global,0,2015-11,20,4758581.81,0.0,0,0,0.750000,0.0,0.0
2,Global,0,2015-12,11,2814717.00,0.0,0,0,0.636364,0.0,0.0
3,Global,0,2016-01,9,1553103.00,0.0,0,0,0.876136,0.0,0.0
4,Global,0,2016-02,24,5505101.00,0.0,0,0,0.750000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
4922,Modhi,101,2015-01,0,0.00,0.0,0,0,0.000000,0.0,0.0
4923,Modhi,102,2015-01,0,0.00,0.0,0,0,0.000000,0.0,0.0
4924,Modhi,103,2015-01,0,0.00,0.0,0,0,0.000000,0.0,0.0
4925,Modhi,104,2015-01,0,0.00,0.0,0,0,0.000000,0.0,0.0
